# Merge dataset of wildfires with nearest weather station
This notebook merges the [Canadian Wildfire Dataset](https://www.kaggle.com/datasets/ulasozdemir/wildfires-in-canada-19502021) from Kaggle and [Canadian Wildland Fire Information System](https://cwfis.cfs.nrcan.gc.ca/downloads/activefires/) with [climate station data](https://api.weather.gc.ca/collections/climate-stations?lang=en) from Environment and Climate Change Canada (ECCC) and Meteorological Service of Canada (MSC) by using the [MSC GeoMet API](https://eccc-msc.github.io/open-data/msc-geomet/readme_en/). To do this, we utilize haversine distance to associate each wildfire to its nearest climate station

In [1]:
import pandas as pd
import numpy as np

## Station data

In [2]:
stations_df = pd.read_csv('stations.csv')

In [3]:
# Convert to standard decimal degrees
stations_df['LATITUDE'] = stations_df['LATITUDE'] / 10000000
stations_df['LONGITUDE'] = stations_df['LONGITUDE'] / 10000000

In [4]:
# Create coordinates column, merges both latitude and longitude with ',' as a seperator
stations_df['COORDINATES'] = stations_df['LATITUDE'].astype(str) + ',' + stations_df['LONGITUDE'].astype(str)

In [5]:
stations_df

,STN_ID,STATION_NAME,PROV_STATE_TERR_CODE,ENG_PROV_NAME,FRE_PROV_NAME,COUNTRY,LATITUDE,LONGITUDE,TIMEZONE,ELEVATION,...,HLY_FIRST_DATE,HLY_LAST_DATE,DLY_FIRST_DATE,DLY_LAST_DATE,MLY_FIRST_DATE,MLY_LAST_DATE,HAS_MONTHLY_SUMMARY,HAS_NORMALS_DATA,HAS_HOURLY_DATA,COORDINATES
0,5679,FIRE LAKE,QC,QUEBEC,QUÉBEC,CAN,52.2000,-67.2200,EST,609.6,...,NaN,NaN,1974-07-01 00:00:00,1974-08-31 00:00:00,1974-01-01 00:00:00,1974-12-01 00:00:00,Y,N,N,"52.2,-67.22"
1,5680,FORESTVILLE,QC,QUEBEC,QUÉBEC,CAN,48.4400,-69.0500,EST,76.2,...,NaN,NaN,1963-01-01 00:00:00,1996-10-31 00:00:00,1963-01-01 00:00:00,1996-10-01 00:00:00,Y,N,N,"48.44,-69.05"
2,5682,FORET MONTMORENCY,QC,QUEBEC,QUÉBEC,CAN,47.1900,-71.0900,EST,640.0,...,NaN,NaN,1965-07-01 00:00:00,2002-01-31 00:00:00,1965-01-01 00:00:00,2002-01-01 00:00:00,Y,N,N,"47.19,-71.09"
3,5687,GRANDES BERGERONNES,QC,QUEBEC,QUÉBEC,CAN,48.1500,-69.3100,EST,61.0,...,NaN,NaN,1951-08-01 00:00:00,2023-02-28 00:00:00,1951-01-01 00:00:00,2015-11-01 00:00:00,Y,Y,N,"48.15,-69.31"
4,5688,GRAND FONDS,QC,QUEBEC,QUÉBEC,CAN,47.4500,-70.0700,EST,365.8,...,NaN,NaN,1968-10-01 00:00:00,1991-05-31 00:00:00,1968-01-01 00:00:00,1991-11-01 00:00:00,Y,N,N,"47.45,-70.07"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8547,61,JAMES ISLAND,BC,BRITISH COLUMBIA,COLOMBIE-BRITANNIQUE,CAN,48.3600,-123.2100,PST,53.6,...,NaN,NaN,1914-02-01 00:00:00,1978-09-30 00:00:00,1914-01-01 00:00:00,1978-12-01 00:00:00,Y,N,N,"48.36,-123.21"
8548,78,SAANICHTON MT NEWTON,BC,BRITISH COLUMBIA,COLOMBIE-BRITANNIQUE,CAN,48.3551,-123.2538,PST,42.7,...,NaN,NaN,1980-08-01 00:00:00,2024-07-12 00:00:00,1980-01-01 00:00:00,2007-02-01 00:00:00,Y,Y,N,"48.3551,-123.2538"
8549,85,SAANICH DOM ASTRO OBS,BC,BRITISH COLUMBIA,COLOMBIE-BRITANNIQUE,CAN,48.3100,-123.2500,PST,222.5,...,NaN,NaN,1916-12-01 00:00:00,1977-06-30 00:00:00,1916-01-01 00:00:00,1977-12-01 00:00:00,Y,N,N,"48.31,-123.25"
8550,87,SAANICHTON CDA,BC,BRITISH COLUMBIA,COLOMBIE-BRITANNIQUE,CAN,48.3718,-123.2508,PST,61.0,...,NaN,NaN,1914-03-01 00:00:00,2023-02-09 00:00:00,1914-01-01 00:00:00,2007-02-01 00:00:00,Y,Y,N,"48.3718,-123.2508"


Review data for stations that are at the same location

In [6]:
# Duplicates in location
duplicates_df = stations_df[stations_df.duplicated(subset='COORDINATES', keep=False)]
duplicates_df = duplicates_df[['STATION_NAME', 'PROV_STATE_TERR_CODE', 'LATITUDE', 'LONGITUDE', 'COORDINATES', 'FIRST_DATE', 'LAST_DATE', 'ELEVATION', 'CLIMATE_IDENTIFIER']]
duplicates_df

,STATION_NAME,PROV_STATE_TERR_CODE,LATITUDE,LONGITUDE,COORDINATES,FIRST_DATE,LAST_DATE,ELEVATION,CLIMATE_IDENTIFIER
5,HAVRE-SAINT-PIERRE A,QC,50.1700,-63.3600,"50.17,-63.36",1983-12-01 00:00:00,2008-07-24 09:00:00,37.8,7043018
32,MURDOCHVILLE HOLLAND,QC,48.5700,-65.3100,"48.57,-65.31",1973-01-01 00:00:00,1974-12-01 00:00:00,624.8,705ECH0
45,ETANG A LA TRUITE,QC,48.4100,-66.4700,"48.41,-66.47",1982-01-01 00:00:00,1983-12-01 00:00:00,337.1,7052C09
59,KIRKLAND LAKE,ON,48.0900,-80.0000,"48.09,-80.0",1950-01-01 00:00:00,1996-12-31 00:00:00,324.0,6074209
64,NEWMARKET,ON,44.0400,-79.2600,"44.04,-79.26",1871-01-01 00:00:00,1967-12-01 00:00:00,281.9,6155616
...,...,...,...,...,...,...,...,...,...
8499,TOFINO A,BC,49.0456,-125.4621,"49.0456,-125.4621",2014-10-23 11:00:00,2024-07-15 12:30:02,24.4,1038210
8500,STEWART A,BC,55.5600,-129.5900,"55.56,-129.59",2015-01-15 11:00:00,2024-07-15 12:30:02,7.3,1067741
8501,BLUE RIVER A,BC,52.0729,-119.1734,"52.0729,-119.1734",2015-01-15 11:00:00,2024-07-15 12:30:02,690.4,1160898
8503,BLACKCOMB MOUNTAIN 1540,BC,50.0600,-122.5500,"50.06,-122.55",1983-01-01 00:00:00,1994-12-01 00:00:00,1540.2,1100Q7Q


Each wildfire can only be associated to a single station. We want to sort the stations by the latest date, and only keep the station with the most recent data for each coordinate point

In [7]:
# Convert LAST_DATE to date
stations_df['LAST_DATE'] = pd.to_datetime(stations_df['LAST_DATE']).dt.date

In [8]:
# Sort from earliest to latest
stations_df = stations_df.sort_values(by='LAST_DATE')

In [9]:
stations_df

,STN_ID,STATION_NAME,PROV_STATE_TERR_CODE,ENG_PROV_NAME,FRE_PROV_NAME,COUNTRY,LATITUDE,LONGITUDE,TIMEZONE,ELEVATION,...,HLY_FIRST_DATE,HLY_LAST_DATE,DLY_FIRST_DATE,DLY_LAST_DATE,MLY_FIRST_DATE,MLY_LAST_DATE,HAS_MONTHLY_SUMMARY,HAS_NORMALS_DATA,HAS_HOURLY_DATA,COORDINATES
608,6776,FORTEAU,NL,NEWFOUNDLAND,TERRE-NEUVE,CAN,51.2800,-56.5800,NST,NaN,...,NaN,NaN,1871-09-01 00:00:00,1878-09-30 00:00:00,1871-01-01 00:00:00,1878-12-01 00:00:00,Y,N,N,"51.28,-56.58"
1456,4770,GRANTON,ON,ONTARIO,ONTARIO,CAN,43.1200,-81.2000,EST,315.5,...,NaN,NaN,1873-01-01 00:00:00,1886-12-31 00:00:00,1873-01-01 00:00:00,1886-12-01 00:00:00,Y,N,N,"43.12,-81.2"
6541,4253,CORNWALL,ON,ONTARIO,ONTARIO,CAN,45.0100,-74.4400,EST,53.3,...,NaN,NaN,1867-01-01 00:00:00,1887-12-31 00:00:00,1867-01-01 00:00:00,1887-12-01 00:00:00,Y,N,N,"45.01,-74.44"
6661,5344,CRANBOURNE,QC,QUEBEC,QUÉBEC,CAN,46.2300,-70.4200,EST,NaN,...,NaN,NaN,1875-07-01 00:00:00,1890-11-30 00:00:00,1875-01-01 00:00:00,1890-12-01 00:00:00,Y,N,N,"46.23,-70.42"
6799,4788,LONDON,ON,ONTARIO,ONTARIO,CAN,42.5900,-81.1200,EST,246.3,...,NaN,NaN,1871-11-01 00:00:00,1891-12-31 00:00:00,1871-01-01 00:00:00,1891-12-01 00:00:00,Y,N,N,"42.59,-81.12"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7760,51077,CLYDE RIVER A,NU,NUNAVUT,NUNAVUT,CAN,70.2909,-68.3101,EST,26.5,...,2013-01-10 11:00:00,2024-07-15 12:30:02,2013-01-10 00:00:00,2024-07-15 00:00:00,NaN,NaN,Y,N,Y,"70.2909,-68.3101"
7759,50822,LA GRANDE RIVIERE A,QC,QUEBEC,QUÉBEC,CAN,53.3731,-77.4215,EST,194.8,...,2012-11-08 10:00:00,2024-07-15 12:30:02,2012-11-08 00:00:00,2024-07-15 00:00:00,NaN,NaN,Y,N,Y,"53.3731,-77.4215"
7758,50722,MUSKRAT DAM,ON,ONTARIO,ONTARIO,CAN,53.2629,-91.4546,EST,277.7,...,2012-11-13 10:00:00,2024-07-15 12:30:02,2012-11-15 00:00:00,2024-07-15 00:00:00,NaN,NaN,Y,N,N,"53.2629,-91.4546"
1357,50090,ROUYN-NORANDA A,QC,QUEBEC,QUÉBEC,CAN,48.1222,-78.5008,EST,301.1,...,2012-12-03 12:00:00,2024-07-15 12:30:02,2018-10-30 00:00:00,2024-07-15 00:00:00,NaN,NaN,Y,N,Y,"48.1222,-78.5008"


In [10]:
# Unique coordinates only, keep only last occurrence of duplicated row (station with most recent data)
unique_stations_df = stations_df.drop_duplicates(subset='COORDINATES', keep='last')

In [11]:
# Drop useless columns
unique_stations_df = unique_stations_df[['STATION_NAME', 'PROV_STATE_TERR_CODE', 'LATITUDE', 'LONGITUDE', 'ELEVATION']]
unique_stations_df

,STATION_NAME,PROV_STATE_TERR_CODE,LATITUDE,LONGITUDE,ELEVATION
608,FORTEAU,NL,51.2800,-56.5800,NaN
1456,GRANTON,ON,43.1200,-81.2000,315.5
6541,CORNWALL,ON,45.0100,-74.4400,53.3
6661,CRANBOURNE,QC,46.2300,-70.4200,NaN
6799,LONDON,ON,42.5900,-81.1200,246.3
...,...,...,...,...,...
7760,CLYDE RIVER A,NU,70.2909,-68.3101,26.5
7759,LA GRANDE RIVIERE A,QC,53.3731,-77.4215,194.8
7758,MUSKRAT DAM,ON,53.2629,-91.4546,277.7
1357,ROUYN-NORANDA A,QC,48.1222,-78.5008,301.1


## Preparing wildfire dataset

In [12]:
wildfire_df = pd.read_csv('2000-2021+2023-2024.csv')

In [13]:
# Drop response type column
wildfire_df.drop('response_type', axis=1, inplace=True)

In [14]:
# Add new column fire_id which will keep track of unique fires after merging
wildfire_df['fire_id'] = wildfire_df.index
wildfire_df

,agency,lat,lon,date,hectares,cause,fire_id
0,BC,50.9050,-126.9292,2000-03-10,6.00,H,0
1,BC,49.2971,-122.2321,2000-08-21,0.20,H,1
2,BC,55.8000,-124.8167,2005-06-20,0.01,L,2
3,BC,49.8770,-121.5730,2000-08-06,0.10,L,3
4,BC,49.0667,-121.8333,2000-07-28,0.10,H,4
...,...,...,...,...,...,...,...
148756,AK,65.0282,-141.2460,2024-05-15,0.00,NaN,148756
148757,BC,55.9370,-121.9170,2024-05-12,295.00,NaN,148757
148758,PC,60.2150,-112.8720,2024-04-26,0.10,NaN,148758
148759,PC,60.1950,-112.7960,2024-04-26,0.10,NaN,148759


In [15]:
wildfire_df

,agency,lat,lon,date,hectares,cause,fire_id
0,BC,50.9050,-126.9292,2000-03-10,6.00,H,0
1,BC,49.2971,-122.2321,2000-08-21,0.20,H,1
2,BC,55.8000,-124.8167,2005-06-20,0.01,L,2
3,BC,49.8770,-121.5730,2000-08-06,0.10,L,3
4,BC,49.0667,-121.8333,2000-07-28,0.10,H,4
...,...,...,...,...,...,...,...
148756,AK,65.0282,-141.2460,2024-05-15,0.00,NaN,148756
148757,BC,55.9370,-121.9170,2024-05-12,295.00,NaN,148757
148758,PC,60.2150,-112.8720,2024-04-26,0.10,NaN,148758
148759,PC,60.1950,-112.7960,2024-04-26,0.10,NaN,148759


In [16]:
# Merging datasets takes too much memory, so chunk down the task in 3
# small_wildfire_df = wildfire_df[:50000]
# small_wildfire_df = wildfire_df[50000:100000]
small_wildfire_df = wildfire_df[100000:]

In [17]:
# Reorganize columns and drop cause
small_wildfire_df = small_wildfire_df[['fire_id', 'lat', 'lon', 'date', 'hectares']]
small_wildfire_df

,fire_id,lat,lon,date,hectares
0,0,50.905000,-126.929200,2000-03-10,6.00
1,1,49.297100,-122.232100,2000-08-21,0.20
2,2,55.800000,-124.816700,2005-06-20,0.01
3,3,49.877000,-121.573000,2000-08-06,0.10
4,4,49.066700,-121.833300,2000-07-28,0.10
...,...,...,...,...,...
49995,49995,52.984067,-115.528783,2013-07-04,0.01
49996,49996,52.917845,-115.339839,2013-08-18,0.01
49997,49997,52.888741,-114.964868,2013-08-21,0.90
49998,49998,52.352750,-114.973983,2013-09-04,0.01


## Haversine distance

In [18]:
# Haversine formula
def haversine(lon1, lat1, lon2, lat2):
    diff_lat = lat2 - lat1
    diff_lon = lon2 - lon1

    haver = np.sin(diff_lat/2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(diff_lon / 2.0) ** 2
    dist = 2 * np.arcsin(np.sqrt(haver))
    
    # Distance in km
    return 6367 * dist

In [19]:
# Cross-joining wildfires
df = small_wildfire_df.merge(unique_stations_df, how='cross')

In [20]:
df

,fire_id,lat,lon,date,hectares,STATION_NAME,PROV_STATE_TERR_CODE,LATITUDE,LONGITUDE,ELEVATION
0,0,50.905000,-126.9292,2000-03-10,6.00,FORTEAU,NL,51.2800,-56.5800,NaN
1,0,50.905000,-126.9292,2000-03-10,6.00,GRANTON,ON,43.1200,-81.2000,315.5
2,0,50.905000,-126.9292,2000-03-10,6.00,CORNWALL,ON,45.0100,-74.4400,53.3
3,0,50.905000,-126.9292,2000-03-10,6.00,CRANBOURNE,QC,46.2300,-70.4200,NaN
4,0,50.905000,-126.9292,2000-03-10,6.00,LONDON,ON,42.5900,-81.1200,246.3
...,...,...,...,...,...,...,...,...,...,...
403449995,49999,52.399467,-114.9698,2013-09-23,0.01,CLYDE RIVER A,NU,70.2909,-68.3101,26.5
403449996,49999,52.399467,-114.9698,2013-09-23,0.01,LA GRANDE RIVIERE A,QC,53.3731,-77.4215,194.8
403449997,49999,52.399467,-114.9698,2013-09-23,0.01,MUSKRAT DAM,ON,53.2629,-91.4546,277.7
403449998,49999,52.399467,-114.9698,2013-09-23,0.01,ROUYN-NORANDA A,QC,48.1222,-78.5008,301.1


In [21]:
# Convert fire and weather climate station to radians using haversine
df[['fire_lat_radians', 'fire_lon_radians']] = np.radians(df.loc[:, ['lat', 'lon']])
df[['clim_lat_radians', 'clim_lon_radians']] = np.radians(df.loc[:, ['LATITUDE', 'LONGITUDE']])

In [22]:
df

,fire_id,lat,lon,date,hectares,STATION_NAME,PROV_STATE_TERR_CODE,LATITUDE,LONGITUDE,ELEVATION,fire_lat_radians,fire_lon_radians,clim_lat_radians,clim_lon_radians
0,0,50.905000,-126.9292,2000-03-10,6.00,FORTEAU,NL,51.2800,-56.5800,NaN,0.888460,-2.215332,0.895005,-0.987507
1,0,50.905000,-126.9292,2000-03-10,6.00,GRANTON,ON,43.1200,-81.2000,315.5,0.888460,-2.215332,0.752586,-1.417207
2,0,50.905000,-126.9292,2000-03-10,6.00,CORNWALL,ON,45.0100,-74.4400,53.3,0.888460,-2.215332,0.785573,-1.299223
3,0,50.905000,-126.9292,2000-03-10,6.00,CRANBOURNE,QC,46.2300,-70.4200,NaN,0.888460,-2.215332,0.806866,-1.229061
4,0,50.905000,-126.9292,2000-03-10,6.00,LONDON,ON,42.5900,-81.1200,246.3,0.888460,-2.215332,0.743336,-1.415811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403449995,49999,52.399467,-114.9698,2013-09-23,0.01,CLYDE RIVER A,NU,70.2909,-68.3101,26.5,0.914543,-2.006602,1.226808,-1.192236
403449996,49999,52.399467,-114.9698,2013-09-23,0.01,LA GRANDE RIVIERE A,QC,53.3731,-77.4215,194.8,0.914543,-2.006602,0.931536,-1.351260
403449997,49999,52.399467,-114.9698,2013-09-23,0.01,MUSKRAT DAM,ON,53.2629,-91.4546,277.7,0.914543,-2.006602,0.929613,-1.596184
403449998,49999,52.399467,-114.9698,2013-09-23,0.01,ROUYN-NORANDA A,QC,48.1222,-78.5008,301.1,0.914543,-2.006602,0.839891,-1.370097


In [23]:
# Create new column containing distance in km
df['distance'] = haversine(df['fire_lon_radians'], df['fire_lat_radians'], df['clim_lon_radians'], df['clim_lat_radians'])

In [24]:
df

,fire_id,lat,lon,date,hectares,STATION_NAME,PROV_STATE_TERR_CODE,LATITUDE,LONGITUDE,ELEVATION,fire_lat_radians,fire_lon_radians,clim_lat_radians,clim_lon_radians,distance
0,0,50.905000,-126.9292,2000-03-10,6.00,FORTEAU,NL,51.2800,-56.5800,NaN,0.888460,-2.215332,0.895005,-0.987507,4714.363978
1,0,50.905000,-126.9292,2000-03-10,6.00,GRANTON,ON,43.1200,-81.2000,315.5,0.888460,-2.215332,0.752586,-1.417207,3510.699347
2,0,50.905000,-126.9292,2000-03-10,6.00,CORNWALL,ON,45.0100,-74.4400,53.3,0.888460,-2.215332,0.785573,-1.299223,3876.030124
3,0,50.905000,-126.9292,2000-03-10,6.00,CRANBOURNE,QC,46.2300,-70.4200,NaN,0.888460,-2.215332,0.806866,-1.229061,4084.900477
4,0,50.905000,-126.9292,2000-03-10,6.00,LONDON,ON,42.5900,-81.1200,246.3,0.888460,-2.215332,0.743336,-1.415811,3546.484725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403449995,49999,52.399467,-114.9698,2013-09-23,0.01,CLYDE RIVER A,NU,70.2909,-68.3101,26.5,0.914543,-2.006602,1.226808,-1.192236,3054.762589
403449996,49999,52.399467,-114.9698,2013-09-23,0.01,LA GRANDE RIVIERE A,QC,53.3731,-77.4215,194.8,0.914543,-2.006602,0.931536,-1.351260,2490.880150
403449997,49999,52.399467,-114.9698,2013-09-23,0.01,MUSKRAT DAM,ON,53.2629,-91.4546,277.7,0.914543,-2.006602,0.929613,-1.596184,1574.526150
403449998,49999,52.399467,-114.9698,2013-09-23,0.01,ROUYN-NORANDA A,QC,48.1222,-78.5008,301.1,0.914543,-2.006602,0.839891,-1.370097,2605.110917


## Create new dataset with unique fire-station pair

In [25]:
# Group by the entire dataset on the basis of fire identity (fire_id) and then select the rows with
# minimum value for distance from weather station
fire_station = df.groupby(['fire_id'])['distance'].min()

In [26]:
fire_stations_df = pd.DataFrame(fire_station, columns=['distance'])
fire_stations_df['fire_id'] = fire_stations_df.index
fire_stations_df.index = range(fire_stations_df.shape[0]) # Make label column

In [27]:
fire_stations_df

,distance,fire_id
0,48.031939,0
1,6.992181,1
2,43.984046,2
3,32.742051,3
4,12.160548,4
...,...,...
49995,11.795653,49995
49996,11.875635,49996
49997,22.851316,49997
49998,15.983157,49998


In [28]:
# Final merge
final_df = pd.merge(df, fire_stations_df, on=['fire_id', 'distance'])

In [29]:
# Drop useless columns
final_df.drop(['fire_lat_radians', 'fire_lon_radians', 'clim_lat_radians', 'clim_lon_radians'], axis=1, inplace=True)

In [30]:
final_df

,fire_id,lat,lon,date,hectares,STATION_NAME,PROV_STATE_TERR_CODE,LATITUDE,LONGITUDE,ELEVATION,distance
0,0,50.905000,-126.929200,2000-03-10,6.00,EGG ISLAND,BC,51.145000,-127.50073,14.0,48.031939
1,1,49.297100,-122.232100,2000-08-21,0.20,PITT LAKE,BC,49.260000,-122.31000,7.6,6.992181
2,2,55.800000,-124.816700,2005-06-20,0.01,MESILINKA CAMP,BC,56.070000,-124.30000,803.0,43.984046
3,3,49.877000,-121.573000,2000-08-06,0.10,LYTTON A,BC,50.132800,-121.34550,224.9,32.742051
4,4,49.066700,-121.833300,2000-07-28,0.10,VEDDER SOUTH LICKMAN,BC,49.060000,-122.00000,15.2,12.160548
...,...,...,...,...,...,...,...,...,...,...,...
49996,49995,52.984067,-115.528783,2013-07-04,0.01,WOLF LO,AB,53.090000,-115.54000,1098.5,11.795653
49997,49996,52.917845,-115.339839,2013-08-18,0.01,BRAZEAU LO,AB,53.010000,-115.25000,1088.4,11.875635
49998,49997,52.888741,-114.964868,2013-08-21,0.90,VIOLET GROVE CS,AB,53.083701,-115.07350,914.0,22.851316
49999,49998,52.352750,-114.973983,2013-09-04,0.01,HORBURG,AB,52.240000,-115.12000,1036.0,15.983157


In [31]:
# final_df.to_csv('fires-merged-1.csv', index=False)
# final_df.to_csv('fires-merged-2.csv', index=False)
final_df.to_csv('fires-merged-3.csv', index=False)

# Merge datasets

In [2]:
# Read
fires_1 = pd.read_csv('fires-merged-1.csv')
fires_2 = pd.read_csv('fires-merged-2.csv')
fires_3 = pd.read_csv('fires-merged-3.csv')

In [3]:
fires_merged_df = pd.concat([fires_1, fires_2, fires_3])

In [4]:
fires_merged_df

,fire_id,lat,lon,date,hectares,STATION_NAME,PROV_STATE_TERR_CODE,LATITUDE,LONGITUDE,ELEVATION,distance
0,0,50.9050,-126.9292,2000-03-10,6.00,EGG ISLAND,BC,51.1450,-127.50073,14.0,48.031939
1,1,49.2971,-122.2321,2000-08-21,0.20,PITT LAKE,BC,49.2600,-122.31000,7.6,6.992181
2,2,55.8000,-124.8167,2005-06-20,0.01,MESILINKA CAMP,BC,56.0700,-124.30000,803.0,43.984046
3,3,49.8770,-121.5730,2000-08-06,0.10,LYTTON A,BC,50.1328,-121.34550,224.9,32.742051
4,4,49.0667,-121.8333,2000-07-28,0.10,VEDDER SOUTH LICKMAN,BC,49.0600,-122.00000,15.2,12.160548
...,...,...,...,...,...,...,...,...,...,...,...
48757,148756,65.0282,-141.2460,2024-05-15,0.00,CLINTON CREEK,YT,64.2800,-140.44000,576.1,91.556968
48758,148757,55.9370,-121.9170,2024-05-12,295.00,HUDSON HOPE BCHPA DAM,BC,56.0100,-122.12000,678.2,15.005006
48759,148758,60.2150,-112.8720,2024-04-26,0.10,BEAR TOWER,NT,60.3100,-113.14000,213.4,18.156877
48760,148759,60.1950,-112.7960,2024-04-26,0.10,BEAR TOWER,NT,60.3100,-113.14000,213.4,22.870821


In [5]:
fires_merged_df.to_csv('fires-merged.csv', index=False)